<a href="https://colab.research.google.com/github/tino926/new_ai_scripts/blob/main/huggingface_diffusers/huggingface_diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref: <https://machinelearningmastery.com/inpainting-and-outpainting-with-diffusers/>  
ref: <https://huggingface.co/docs/diffusers/using-diffusers/inpaint>  
ref: <https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers_doc/en/inpaint.ipynb#scrollTo=9Apdxcp_-kA3>  
ref: <https://medium.com/@nimritakoul01/image-inpainting-using-stable-diffusion-from-hugging-face-d68a26da9fd2>  

In [ ]:
import numpy as np
import torch
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow

# Install the 'diffusers' library for working with diffusion models
# (e.g., Stable Diffusion)
!pip install diffusers

# Install the Segment Anything library, which is a segmentation model
# by Facebook Research
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'
from segment_anything import sam_model_registry, SamPredictor

!pip install diffusers accelerate
from diffusers import StableDiffusionInpaintPipeline

!wget -q -nc https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
CHECKPOINT_PATH='/content/sam_vit_b_01ec64.pth'

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_b"

In [ ]:
print(DEVICE)

In [ ]:
!wget https://machinelearningmastery.com/wp-content/uploads/2024/04/inpaint-example.png

In [ ]:
# Give the path of your image
IMAGE_PATH = '/content/inpaint-example.png'
# Read the image from the path
image = cv2.imread(IMAGE_PATH)
cv2_imshow(image)
# Convert to RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [ ]:
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)
mask_predictor = SamPredictor(sam)
mask_predictor.set_image(image_rgb)

# Provide points as input prompt [X, Y]-coordinates
input_point = np.array([[250, 250]])
input_label = np.array([1])

# Predicting Segmentation mask
masks, scores, logits = mask_predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=False,
)

In [ ]:
mask = masks.astype(float) * 255
mask = np.transpose(mask, (1, 2, 0))
_ , bw_image = cv2.threshold(mask, 100, 255, cv2.THRESH_BINARY)
cv2_imshow(bw_image)
cv2.imwrite('mask.png', bw_image)
del sam, mask_predictor   # delete models to conserve GPU memory

In [ ]:
# !pip install --upgrade transformers diffusers

In [ ]:
# Load images using PIL
init_image = Image.open(IMAGE_PATH)
mask_image = Image.open('mask.png')

import requests
import PIL
from io import BytesIO
def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")


img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = download_image(img_url).resize((512, 512))
mask_image = download_image(mask_url).resize((512, 512))


pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
)
pipe = pipe.to(DEVICE)

In [ ]:
prompt = "red roses in a tall vase"
image = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
# convert image to opencv mat
image.save('output.png')

image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
cv2_imshow(image_cv)
